# How to query the PAP Knowledge base: a jupyter notebook guide

This guide assumes you have a fuseki-server running on http://127.0.0.1:3030.

Please refer to the [README](https://github.com/mobr-ai/PolkadotAnalytics/blob/main/README.md) file to understand how to run this service.

The KBM class creates an abstraction layer on top of our triplestore's SPARQL engine. A triplestore is a database used to store and manage semantic data in the form of subject-predicate-object triples. The triplestore along with PAP endpoints and the POnto ontology form the basis of PAP knowledge base. SPARQL is the W3C standard language to query triplestores.

The first step to query PAP's knowledge base is importing the KBM (Knowledge Base Manager) class. 

```python
from pap.datalayer.kbm import KBM
```

Then we can make a query using the method 

```python
KBM.run_sparql(sparql_spec:str, filter:str)
```

In a sparql query usually you specify wich term you are interested in the SELECT statement, having an '*' when you are interested in all the terms in the triple. The second parameter in the KBM method specifies a filter if you only want a specific term from the query that you are interested, so the results will be retrieved accordingly. 

The code below executes a query to get all the Parachains described in the Knowledge Base. The query is specified in the sparql_spec variable. For each query it is key to specify the prefixes you will use in the query. The variable KBM.sparql_prefix keeps POnto prefixes for us so we can reuse them in our queries, while being able to add others as needes. Details about the SPARQL query language can be found [here](https://www.w3.org/TR/sparql11-query/).

Note that the result brings only a few Parachains, which we created only to illustrate the process. The appropriate representation will be available only after the PAP stage 2 is complete. The primary goal of stage 2 will be to structure and develop processing workflows to gather raw data from the Polkadot Ecosystem. The knowledge base will dynamically maintain a formal representation of extracted data aligned with the POnto ontology.

> Tip: use the code snippets below to test your own queries

In [1]:
import sys
sys.path.append("../")

from pap.datalayer.kbm import KBM

sparql_spec = KBM.sparql_prefix + """
SELECT ?s
WHERE {
    ?s a ponto:Parachain
}
"""

r = KBM.run_sparql(sparql_spec, "s")
print (r)

['Collectives', 'Picasso', 'Moonbeam', 'Ganshiro', 'Phala', 'Acala', 'Statemint', 'Astar', 'Moonriver']


This type of query can be specified for any class represented in the knowledge base. The code snippet below gets a list of all the POnto classes present in the KB.

In [2]:
sparql_spec = KBM.sparql_prefix + """
    SELECT DISTINCT ?s
    WHERE {
    {
        ?s a owl:Class .
    }
    UNION
    {
        ?s rdfs:subClassOf ?class .
    }
    }
"""

r = KBM.run_sparql(sparql_spec, "s")
print (r)

['MultisigAccount', 'Query', 'DutyRoster', 'Component', 'ProofOfStake', 'ConsensusMechanism', 'LockedBalance', 'Wallet', 'Proposal', 'ReservationReason', 'ProofOfWork', 'KSM', 'Equivocation', 'Module', 'ExternalQueue', 'HardSpoon', 'StakeHolder', 'Babe', 'Collator', 'InjectedAccount', 'Metadata', 'RuntimeModule', 'FinalityGadget', 'Treasury', 'BLS', 'BrowserWallet', 'Account', 'Consumers', 'WhitelistPallet', 'OpenGov', 'Beneficiary', 'XCMChannel', 'RootOrigin', 'SDK', 'ProofOfValidity', 'Inherent', 'Fellowship', 'Collations', 'Candidate', 'VotingPower', 'Validator', 'GovernanceV1', 'Oracle', 'FRAME', 'CommunityQueue', 'Remarks', 'Referendum', 'TechnicalCommittee', 'Teleport', 'AccountHolder', 'Block', 'Asset', 'Dapp', 'Providers', 'LockType', 'FullNode', 'SlotLeader', 'RandomSeed', 'DemocracyLockType', 'Node', 'FungibleToken', 'Substrate', 'StakePool', 'TVL', 'AvailabilityCores', 'Parachain', 'NativeToken', 'Epoch', 'Origin', 'Crowdloan', 'Parathread', 'EpochTimeSlot', 'ParachainRegist

The way POnto ontology is structure, each class has its deffinition specified as an rdfs:comment. Note that any POnto class can be used in the query specified in the code below to answer "what is" type of questions.

In [3]:
sparql_spec = KBM.sparql_prefix + """
    SELECT DISTINCT ?def
    WHERE {
        ponto:XCMChannel rdfs:comment ?def
    }
"""

r = KBM.run_sparql(sparql_spec, "def")
print (r)

['Represents a communication pathway between two or more Parachains or between a Parachain and the Relay Chain. It allows for the exchange of messages, instructions, and assets across different chains, facilitating inter-chain communication.']


Now lets create some individuals to illustrate how powerful a query can be. It is important to highlight that these individuals will be automatically fetched from the Polkadot ecosystem as soon as PAP stage 2 is completed.

In the example, the triples specifies three accounts a1, a2, and a3. These accounts have 6.1, 4.9, and 5.2 DOTs, respectivelly. In addition, the account a3 has 5.1 KSM in it.

In the code snippet below, the triples are stored in the triples variable, which is used in the KBM.inject_triples method to inject the triples in the knowledge base.

In [4]:
triples = """
     ponto:a1 a ponto:Account ;
          ponto:hasToken ponto:dot1 .

     ponto:a2 a ponto:Account ;
          ponto:hasToken ponto:dot2 .

     ponto:a3 a ponto:Account ;
          ponto:hasToken ponto:dot3 ;
          ponto:hasToken ponto:ksm1 .

     ponto:tb1 a ponto:TransferableBalance ;
          ponto:hasBalanceValue "6.1"^^xsd:decimal .

     ponto:tb2 a ponto:TransferableBalance ;
          ponto:hasBalanceValue "4.9"^^xsd:decimal .

     ponto:tb3 a ponto:TransferableBalance ;
          ponto:hasBalanceValue "5.2"^^xsd:decimal .

     ponto:tb4 a ponto:TransferableBalance ;
          ponto:hasBalanceValue "5.1"^^xsd:decimal .

     ponto:dot1 a ponto:DOT ;
          ponto:hasTransferableBalance ponto:tb1 .

     ponto:dot2 a ponto:DOT ;
          ponto:hasTransferableBalance ponto:tb2 .

     ponto:dot3 a ponto:DOT ;
          ponto:hasTransferableBalance ponto:tb3 .

     ponto:ksm1 a ponto:KSM ;
          ponto:hasTransferableBalance ponto:tb4 .

     ponto:rb1 a ponto:ReservedBalance ;
          ponto:hasBalanceValue "6.1"^^xsd:decimal .

     ponto:arb1 a ponto:Account ;
          ponto:hasAddress "14UgDLpYd8E28QEmnaT4ZJTsM2kcm84Zo2EoD7wDv8S5CE4w"^^xsd:string ;
          ponto:hasReservedBalance ponto:rb1 .

     ponto:ac1 a ponto:Account ;
          ponto:hasReserved ponto:rb1 .

     ponto:tr1 a ponto:Transaction ;
          ponto:hasReserved ponto:rb1 .
"""

KBM.inject_triples(triples)

Now lets use these triples to make a more interesting query. The code snipet below specifies a SPARQL query to check how many accounts have DOT and KSM tokens? 

Note that the result is upon the triples we just injected in the triplestore, for illustrative purposes only. The appropriate representation will be available only after the PAP stage 2 is complete.

In [5]:
sparql_spec = KBM.sparql_prefix + """
    SELECT (COUNT(DISTINCT ?account) AS ?numOfAccounts)
    WHERE {
        ?account ponto:hasToken ?tokenD .
        ?tokenD a ponto:DOT .
        ?account ponto:hasToken ?tokenK .
        ?tokenK a ponto:KSM .
    }
"""

r = KBM.run_sparql(sparql_spec, "numOfAccounts")
print (r)

[1]


In the next code snippets, we illustrate how to specify queries using a query example described in the Substrate-ETL [medium post](https://colorfulnotion.medium.com/polkaholic-ios-2022-xcm-transfers-in-bigquery-public-dataset-substrate-etl-polkadot-xcmtransfers-dfa6f2261ce9).

The triples states a few XCM transfers between the three accounts created above. 

In [6]:

triples = """
    ponto:s1 a ponto:Sufficients ;
      ponto:hasRefCountValue 10 .
    ponto:a4 a ponto:Account ;
      ponto:hasSufficients ponto:s1 .
    ponto:p1 a ponto:Pallet ;
      ponto:dependsOn ponto:s1 .
    ponto:tr2 a ponto:Transaction ;
      ponto:incrementsRefCount ponto:s1 .
    ponto:s1 ponto:hasRefCountValue 20 .

    ponto:bp1 a ponto:Block ;
      ponto:composes ponto:Polkadot .

    ponto:bk1 a ponto:Block ;
      ponto:composes ponto:Kusama .

    ponto:tx1 a ponto:XCMTransfer ;
      ponto:recordedOn ponto:bp1 ;
      ponto:hasOriginationTimestamp "2022-10-01T00:00:00Z"^^xsd:dateTimeStamp ;
      ponto:hasSender ponto:a1 ;
      ponto:hasTransferBeneficiary ponto:a2 ;
      ponto:hasDestinationExecutionStatus "success"^^xsd:string .

    ponto:tx2 a ponto:XCMTransfer ;
      ponto:recordedOn ponto:bp1 ;
      ponto:hasOriginationTimestamp "2022-10-02T00:00:00Z"^^xsd:dateTimeStamp ;
      ponto:hasSender ponto:a3 ;
      ponto:hasTransferBeneficiary ponto:a2 ;
      ponto:hasDestinationExecutionStatus "success"^^xsd:string .

    ponto:tx3 a ponto:XCMTransfer ;
      ponto:recordedOn ponto:bk1 ;
      ponto:hasOriginationTimestamp "2022-11-02T00:00:00Z"^^xsd:dateTimeStamp ;
      ponto:hasSender ponto:a2 ;
      ponto:hasTransferBeneficiary ponto:a1 ;
      ponto:hasDestinationExecutionStatus "success"^^xsd:string .
"""

KBM.inject_triples(triples)

The following code snippet specifies a SPARQL query to answer the question: How many distinct XCM Transfer senders and beneficiaries were there in 2022?

Note that since it was not specified, the query considers transfers between any blockchains represented in the Knowledge Base.

In [7]:
#How many distinct XCM Transfer senders and beneficiaries were there in 2022?

sparql_spec = KBM.sparql_prefix + """
    SELECT (COUNT(DISTINCT ?sender) AS ?senders) (COUNT(DISTINCT ?beneficiary) AS ?beneficiaries) {
        ?transfer a ponto:XCMTransfer .
        ?transfer ponto:hasOriginationTimestamp ?timestamp .
        ?transfer ponto:hasSender ?sender .
        ?transfer ponto:hasTransferBeneficiary ?beneficiary .
        ?transfer ponto:hasDestinationExecutionStatus "success"^^xsd:string .
        FILTER(?timestamp > "2022-01-01T00:00:00Z"^^xsd:dateTimeStamp && ?timestamp < "2022-12-31T23:59:59Z"^^xsd:dateTimeStamp)
    }
"""

r = KBM.run_sparql(sparql_spec)
print (r)

[{'senders': 3, 'beneficiaries': 2}]


The code snippet below presents a SPARQL query to answer the question "How many distinct XCM Transfer senders and beneficiaries were there in 2022 in Polkadot?"

Note that this query could be easily changed to retrieve data from Kusama, by replacing ponto:Polkadot to ponto:Kusama.

In [8]:
#How many distinct XCM Transfer senders and beneficiaries were there in 2022 in Polkadot vs Kusama?
sparql_spec = KBM.sparql_prefix + """
SELECT ?relaychain (COUNT(DISTINCT ?sender) AS ?senders) (COUNT(DISTINCT ?beneficiary) AS ?beneficiaries) {
        ?transfer ponto:recordedOn ?block .
        { ?block ponto:composes ponto:Polkadot . BIND (ponto:Polkadot as ?relaychain) }
        UNION
        { ?block ponto:composes ponto:Kusama . BIND (ponto:Kusama as ?relaychain) }
        ?transfer a ponto:XCMTransfer .
        ?transfer ponto:hasOriginationTimestamp ?timestamp .
        ?transfer ponto:hasSender ?sender .
        ?transfer ponto:hasTransferBeneficiary ?beneficiary .
        ?transfer ponto:hasDestinationExecutionStatus "success"^^xsd:string .
        FILTER(?timestamp > "2022-01-01T00:00:00Z"^^xsd:dateTimeStamp && ?timestamp < "2022-12-31T23:59:59Z"^^xsd:dateTimeStamp)
}
GROUP BY ?relaychain
"""

r = KBM.run_sparql(sparql_spec)
print (r)

[{'relaychain': 'Kusama', 'senders': 1, 'beneficiaries': 1}, {'relaychain': 'Polkadot', 'senders': 2, 'beneficiaries': 1}]


You can also use the KBM class to delete triples. The code snippet below deletes a few of the triples we created in the examples above.

In [9]:
KBM.delete_entity("a1")
KBM.delete_entity("a2")
KBM.delete_entity("a3")
KBM.delete_entity("dot1")
KBM.delete_entity("dot2")
KBM.delete_entity("dot3")
KBM.delete_entity("ksm1")

sparql_spec = KBM.sparql_prefix + """
    SELECT (COUNT(DISTINCT ?account) AS ?numOfAccounts)
    WHERE {
        ?account ponto:hasToken ?tokenD .
        ?tokenD a ponto:DOT .
        ?account ponto:hasToken ?tokenK .
        ?tokenK a ponto:KSM .
    }
"""
r = KBM.run_sparql(sparql_spec, "numOfAccounts")
print (r)

[0]


PAP functionalities also allows you to reset all the representation... like a "reset your device to factory settings" functionality. The code snippet below uses the DatasetManager class to execute a reset by deleting the POnto dataset and creating it again with the initial knowledge representation.

In [10]:
from pap.datalayer.dataset import DatasetManager

dm = DatasetManager()
dm.delete_dataset("POnto")
dm.create_ponto_dataset()

if dm.has_dataset("POnto"):
    print("all done")

all done
